---
title: Low Rank Approximation and the SVD
jupyter: python3
---

::: {.content-hidden when-profile="slides"}
Today, we move on.   

However, let's look back and try to put the modeling we've done into a larger context.
:::

## Models are simplifications

[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tools4ds/DS701-Course-Notes/blob/main/ds701_book/jupyter_notebooks/10-Low-Rank-and-SVD.ipynb)

One way of thinking about modeling or clustering is that we are building a __simplification__ of the data. 

That is, a model is a description of the data, that is simpler than the data.

In particular, instead of thinking of the data as thousands or millions of individual data points, we might think of it in terms of a small number of clusters, or a parametric distribution, etc.

From this simpler description, we hope to gain __insight.__

There is an interesting question here:  __why__ does this process often lead to insight?   

That is, why does it happen so often that a large dataset can be described in terms of a much simpler model?

I don't know.

::: {.content-visible when-profile="slides"}
## Occam's Razor
:::

:::: {.columns}
::: {.column width="40%"}
![](figs/L10-William-of-Ockham.png){width=80%}

[Source](https://commons.wikimedia.org/w/index.php?curid=5523066)
:::
::: {.column width="60%"}
However, I think that William of Ockham (c. 1300 AD) was on the right track.

:::: {.fragment}
He said:

> Non sunt multiplicanda entia sine necessitate
::::

:::: {.fragment}
or, in other words:

> Entities must not be multiplied beyond necessity.
::::

:::: {.fragment}
by which he meant:

> Among competing hypotheses, the one with the fewest assumptions should be selected.
::::


::: {.content-visible when-profile="web"}
Which has come to be known as "Occam's razor."
:::
:::
::::

::: {.content-visible when-profile="slides"}
## Occam's razor
:::

William was saying that it is more common for a set of observations to be determined by a simple process than a complex process.

:::: {.fragment}
In other words, the world is full of simple (but often hidden) patterns.
::::

:::: {.fragment}
From which one can justify the observation that "modeling works suprisingly often."
::::

## Data Matrices

Now we'll consider a (seemingly) very different approximation of data, applicable to data when it is in matrix form. Let $A\in\mathbb{R}^{m\times n}$

$$
A =
\begin{bmatrix}
a_{11} & \cdots & a_{1j} & \cdots & a_{1n} \\
\vdots & \ddots & \vdots & \ddots & \vdots \\
a_{i1} & \cdots & a_{ij} & \cdots & a_{1n} \\
\vdots & \ddots & \vdots & \ddots & \vdots \\
a_{m1} & \cdots & a_{mj} & \cdots & a_{mn} \\
\end{bmatrix}.
$$

The $m$ rows are the data objects and the $n$ columns are the features.

::: {.content-visible when-profile="slides"}
## Data matrix examples
:::

<table>
<tr><th>Data Type</th><th>Rows</th><th>Columns</th><th>Elements</th></tr>
<tr><td>Network Traffic<td>Sources</td><td>Destinations</td><td>Number of Bytes</td></tr>
<tr><td>Social Media</td><td>Users</td><td>time bins</td><td>Number of Posts/Tweets/Likes</td></tr>
<tr><td>Web Browsing</td><td>Users</td><td>Content Categories</td><td>Visit Counts/Bytes Downloaded</td></tr>
<tr><td>Web Browsing</td><td>Users</td><td>time bins</td><td>Visit Counts/Bytes Downloaded</td></tr>
</table>

## Matrix Rank

::: {.content-visible when-profile="web"}
Let's briefly review some definitions.
:::

We'll consider a real matrix $A\in\mathbb{R}^{m\times n}$ with $m>n$.

:::: {.fragment}
The __rank__ of $A$ is the number of linearly independent rows or columns of the matrix. 
::::

:::: {.fragment}
The largest value that a matrix rank can take is $\min(m,n)$. Since we assumed $m>n$, the largest value of the rank is $n$.
::::

:::: {.fragment}
If the matrix $A$ has rank equal to $n$, then we say it is full rank.
::::

:::: {.fragment}
However, it can happen that the rank of a matrix is __less__ than $\min(m,n)$. In this case we say that $A$ is rank-deficient.
::::

::: {.content-visible when-profile="slides"}
## Matrix Rank continued
:::

The dimension of a vector space is the smallest number of (linearly independent) vectors needed to span the space.

:::: {.fragment}
The dimension of the column space of $A$ is the __smallest number of vectors that suffice to construct the columns of $A$.__
::::

:::: {.fragment}
If the dimension of the column spaces is $p$, then there exists a set of vectors $\{\mathbf{u}_1, \mathbf{u}_2, \dots, \mathbf{u}_p\}$ such that every column $\mathbf{a}_i$ can be expressed as:

$$\mathbf{a}_i = c_{i1}\mathbf{u}_1 + c_{i2}\mathbf{u}_2 + \dots + c_{ip}\mathbf{u}_p\quad i=1,\ldots,n.$$
::::

::: {.content-visible when-profile="slides"}
## Matrix Rank continued
:::

Now to store a matrix $A \in \mathbb{R}^{m\times n}$ we need to store $mn$ values.

However, if $A$ has rank $k$, it can be factored as $A = UV$,
$$
\begin{bmatrix}
a_{11} & \cdots & a_{1j} & \cdots & a_{1n} \\
\vdots & \ddots & \vdots & \ddots & \vdots \\
a_{i1} & \cdots & a_{ij} & \cdots & a_{1n} \\
\vdots & \ddots & \vdots & \ddots & \vdots \\
a_{m1} & \cdots & a_{mj} & \cdots & a_{mn} \\
\end{bmatrix} =
\begin{bmatrix}
\vdots &  \vdots  \\
\vdots &  \vdots  \\
\mathbf{u}_1 & \mathbf{u}_{k} \\
\vdots& \vdots  \\
\vdots &  \vdots  \\
\end{bmatrix}
\begin{bmatrix}
\cdots & \cdots & \mathbf{v}_1 & \cdots  & \cdots  \\
\cdots & \cdots & \mathbf{v}_k & \cdots  & \cdots  \\
\end{bmatrix}
$$

where $U \in \mathbb{R}^{m\times k}$ and $V \in \mathbb{R}^{k \times n}$.

This only requires $k(m+n)$ values, which could be much smaller than $mn$.

## Low Effective Rank

In many situations we may wish to __approximate__ a data matrix $A$ with a low-rank matrix $A^{(k)}.$

:::: {.fragment}
To talk about when one matrix "approximates" another, we need a norm for matrices.  
::::

:::: {.fragment}
We will use the __Frobenius norm__, whic his defined as

$$\Vert A\Vert_F = \sqrt{\sum a_{ij}^2}.$$
::::

:::: {.fragment}
Observe that this is the $\ell_2$ norm for a vectorized matrix, i.e., by  stacking the columns of the matrix $A$ to form a vector of length $mn$. 
::::

::: {.content-visible when-profile="slides"}
## Low Effective Rank continued
:::

To quantify when one matrix is "close" to another, we define the distance function:

$$\mbox{dist}(A,B) = \Vert A-B\Vert_F.$$

:::: {.fragment}
This can be viewed as Euclidean distance between $mn$-dimensional vectors (vectorized matrices of dimension $m\times n$).
::::

:::: {.fragment}
We define the __rank-$k$ approximation__ to $A$ as follows.

Let $k < \operatorname{Rank} A$, then the rank-$k$ approximation to $A$ is 

$$A^{(k)} =\mathop{\arg\min}\limits_{\{B~|~\operatorname{Rank} B = k\}} \Vert A-B\Vert_F.$$
::::

:::: {.fragment}
This means that $A^{(k)}$ is the closest rank-$k$ matrix to $A$, or, alternatively, the best rank-$k$ approximation to $A$ in a least-squares sense.
::::

::: {.content-visible when-profile="slides"}
## Low Effective Rank continued
:::

Let's say we have $A^{(k)}$, the rank-$k$ approximation to $A$.  

:::: {.fragment}
By definition, there is a set $~\mathcal{U}$ consisting of $k$ vectors such that each column of $A^{(k)}$ can be expressed as a linear combination of vectors in $~\mathcal{U}$.  
::::
 
:::: {.fragment}
Let us call the matrix formed by those vectors $U$.
::::

:::: {.fragment}
We have that 

$$A^{(k)} = UV^T,$$

for some set of coefficients $V^T$ that describe the linear combinations of $U$ that yield the columns of $A^{(k)}$. 
::::

:::: {.fragment}
So $U$ is $m\times k$ and $V$ is $n\times k$.
::::

::: {.content-visible when-profile="slides"}
## Low Effective Rank continued
:::
If we approximate $A$ by $A^{(k)}$, then the error we incur is:

$$\Vert A-A^{(k)}\Vert_F.$$

:::: {.fragment}
Hence, a rank-$k$ approximation $A^{(k)}$ is valuable if 

:::: {.incremental}
* $\Vert A-A^{(k)}\Vert_F$ is small compared to $\Vert A\Vert_F$, and 
* $k$ is small compared to $m$ and $n$.
::::
::::

:::: {.fragment}
In that case we have achieved a simplification of the data without a great loss in accuracy.
::::

## Finding Rank-$k$ Approximations

There is a celebrated method for finding the best rank-$k$ approximation to any matrix: the __Singular Value Decomposition (SVD).__

:::: {.fragment}
> SVD is "the Rolls-Royce and the Swiss Army Knife of Numerical Linear Algebra.”

Dianne O’Leary, MMDS ’06
::::

:::: {.fragment}
Recall from our [linear algebra refresher](04-Linear-Algebra-Refresher.qmd) that for any matrix $A\in\mathbb{R}^{m\times n}$ with $m>n$, then $A$ admits a decomposition

$$
A = U\Sigma V^{T}.
$$
::::


::: {.content-visible when-profile="slides"}
## SVD
:::

In particular, the singular value decomposition of a rank-$r$ matrix $A$ has the form:

$$
\underbrace{\begin{bmatrix}
\vdots & \vdots &  & \vdots \\
\mathbf{a}_{1} & \mathbf{a}_{2} & \cdots & \mathbf{a}_{n} \\
\vdots & \vdots &  & \vdots \\
\end{bmatrix}}_{A} =
\underbrace{\begin{bmatrix}
\vdots &  \vdots  \\
\mathbf{u}_1 & \mathbf{u}_{r} \\
\vdots& \vdots  \\
\end{bmatrix}}_{U}
\underbrace{\begin{bmatrix}
\sigma_{1} & &    \\
 & \ddots &    \\
&  & \sigma_{r}  \\
\end{bmatrix}}_{\Sigma}
\underbrace{\begin{bmatrix}
\cdots & \mathbf{v}_1 & \cdots   \\
\cdots & \mathbf{v}_r & \cdots   \\
\end{bmatrix}}_{V^T}
$$

:::: {.fragment}
where 

:::: {.incremental}
1. $U$ is $m\times r$
2. The columns of $U$ are mutually orthogonal and unit length, i.e., $U^TU = I$.
3. $V$ is $n\times r$.
4. The columns of $V$ are mutually orthogonal and unit length, i.e., $V^TV = I$.
5. The matrix $\Sigma$ is an $r\times r$ diagonal matrix, whose diagonal values are $\sigma_1 \geq \sigma_2 \geq \dots \geq \sigma_r > 0$.
::::
::::

::: {.content-visible when-profile="slides"}
## SVD continued
:::

The SVD is _incredibly useful_ for finding matrix approximations.

In particular, for an $m\times n$ matrix $A$, the SVD does two things:

:::: {.incremental}
1. It gives the best rank-$k$ approximation to $A$ for __every__ $k$ up to the rank of $A$.
2. It gives the __distance__ of the best approximation $A^{(k)}$ from $A$ for each $k$.
::::

:::: {.fragment}
In terms of the singular value decomposition, the best rank-$k$ approximation to $A$ is formed by taking 

:::: {.incremental}
* $U'$ are the $k$ leftmost columns of $U$, 
* $\Sigma'$ is the $k\times k$ upper left submatrix of $\Sigma$, and 
* $V'$ is the $k$ leftmost columns of $V$, and constructing 
::::

:::: {.fragment}
$$ A^{(k)} = U'\Sigma'(V')^T.$$
::::
::::

::: {.content-visible when-profile="slides"}
## SVD continued
:::

Furthermore, the distance (in Frobenius norm) of the best rank-$k$ approximation $A^{(k)}$ from $A$ is equal to $\sqrt{\sum_{i=k+1}^r\sigma^2_i}$.

That is, if you construct $A^{(k)}$ as shown above, then:

$$\Vert A-A^{(k)}\Vert_F^2 = \sum_{i=k+1}^r\sigma^2_i.$$

:::: {.fragment}
Try to prove this yourself!
::::

## Low Effective Rank of Data Matrices

In general, a data matrix $A\in\mathbb{R}^{m\times n}$  is usually __full rank__, meaning that $\operatorname{Rank} A = \min(m, n)$.

:::: {.fragment}
However, it is possible to encounter data matrices that have __low effective rank.__
::::

:::: {.fragment}
By this we mean that one can usefully approximate $A$ by some $A^{(k)}$ for which $k \ll \min(m,n)$.
::::

:::: {.fragment}
For any data matrix, we can judge when this is the case by looking at its singular values, because the singular values tell us the distance to the nearest rank-$k$ matrix.
::::

## Empirical Evidence

Let's see how this theory can be used in practice  and investigate some real data.

We'll look at data traffic on the Abilene network:

![](figs/L10-Abilene-map.png)

Source: Internet2, circa 2005

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

with open('data/net-traffic/AbileneFlows/odnames','r') as f:
    odnames = [line.strip() for line in f]
dates = pd.date_range('9/1/2003', freq = '10min', periods = 1008)
Atraf = pd.read_table('data/net-traffic/AbileneFlows/X', sep='  ', header=None, names=odnames, engine='python')
Atraf.index = dates
Atraf

::: {.content-visible when-profile="slides"}
## Empirical Evidence continued
:::

In [ ]:
#| code-fold: false
Atraf.shape

As we would expect, our traffic matrix has rank 121:

In [ ]:
#| code-fold: false
np.linalg.matrix_rank(Atraf)

However -- perhaps it has low __effective__ rank.

The `numpy` routine for computing the SVD is `np.linalg.svd`:

In [ ]:
#| code-fold: false
u, s, vt = np.linalg.svd(Atraf)

::: {.content-visible when-profile="slides"}
## Empirical Evidence continued
:::

Now let's look at the singular values of `Atraf` to see if it can be usefully approximated as a low-rank matrix:

In [ ]:
fig = plt.figure(figsize=(5, 3))
plt.plot(range(1, 1+len(s)), s)
plt.xlabel(r'$k$', size=20)
plt.ylabel(r'$\sigma_k$', size=20)
plt.ylim(ymin=0)
plt.xlim(xmin=-1)
plt.title(r'Singular Values of $A$', size=20)
plt.show()

This classic, sharp-elbow tells us that a few singular values are very large, and most singular values are quite small.

::: {.content-visible when-profile="slides"}
## Empirical Evidence continued
:::

Zooming in for just small $k$ values, we can see that the elbow is around 4 - 6 singular values:

In [ ]:
fig = plt.figure(figsize=(5, 3))
Anorm = np.linalg.norm(Atraf)
plt.plot(range(1, 21), s[0:20]/Anorm, '.-')
plt.xlim([0.5, 20])
plt.ylim([0, 1])
plt.xlabel(r'$k$', size=20)
plt.xticks(range(1, 21))
plt.ylabel(r'$\sigma_k$', size=20);
plt.title(r'Singular Values of $A$',size=20)
plt.show()

This pattern of singular values suggests __low effective rank.__

::: {.content-visible when-profile="slides"}
## Empirical Evidence continued
:::

Let's use the formula above to compute the relative error of a rank-$k$ approximation to $A$:

In [ ]:
fig = plt.figure(figsize=(5, 3))
Anorm = np.linalg.norm(Atraf)
err = np.cumsum(s[::-1]**2)
err = np.sqrt(err[::-1])
plt.plot(range(0, 20), err[:20]/Anorm, '.-')
plt.xlim([0, 20])
plt.ylim([0, 1])
plt.xticks(range(1, 21))
plt.xlabel(r'$k$', size = 16)
plt.ylabel(r'relative F-norm error', size=16)
plt.title(r'Relative Error of rank-$k$ approximation to $A$', size=16)
plt.show()

Remarkably, we are down to 9% relative error using only a rank 20 approximation to $A$.

::: {.content-visible when-profile="slides"}
## Empirical Evidence continued
:::

So instead of storing 

* $mn =$ (1008 $\cdot$ 121) = 121,968 values, 

we only need to store 

* $k(m+n)$ = 20 $\cdot$ (1008 + 121) = 22,580 values, 

which is an 81% reduction in size.

## Low Effective Rank is Common

In practice __many__ datasets have low effective rank.   

We consider the following examples:

:::: {.incremental}
* Likes on Facebook,
* Yelp reviews and Tweets (the site formerly known as twitter),
* User preferences over time,
* Images.
::::

::: {.content-visible when-profile="slides"}
## Likes on Facebook
:::

::: {.content-visible when-profile="web"}
__Likes on Facebook.__
:::

Here, the matrices are 

:::: {.incremental}
1. Number of likes:  Timebins $\times$ Users
2. Number of likes:  Users $\times$ Page Categories
3. Entropy of likes across categories:  Timebins $\times$ Users
::::

:::: {.fragment}
![](figs/L10-facebook.png){fig-align="center"}

Source: [Viswanath et al., Usenix Security, 2014]
::::

::: {.content-visible when-profile="slides"}
## Social Media Activity
:::

::: {.content-visible when-profile="web"}
__Social Media Activity.__
:::


Here, the matrices are 

:::: {.incremental}
1. Number of Yelp reviews:  Timebins $\times$ Users
2. Number of Yelp reviews:  Users $\times$ Yelp Categories
3. Number of Tweets:  Users $\times$ Topic Categories
::::

:::: {.fragment}
![](figs/L10-yelp-twitter.png)

Source: [Viswanath et al., Usenix Security, 2014]
::::


::: {.content-visible when-profile="slides"}
## User preferences over items
:::

::: {.content-visible when-profile="web"}
__User preferences over items.__
:::

Example: the Netflix prize worked with partially-observed matrices like this:

$$
\begin{bmatrix}
 & & & \vdots & & & \\
 & & 3 & 2 & & 1 &\\
 & 1 & & 1 & & & \\
\dots & & 2 & & 4 & & \dots\\
 & 5 & 5 & & 4 & & \\
 & 1 & & & 1 & 5 & \\
 & & & \vdots & & & \\
\end{bmatrix},
$$

:::: {.fragment}
where the rows correspond to users, the columns to movies, and the entries are ratings.
::::

:::: {.fragment}
Although the problem matrix was of size 500,000 $\times$ 18,000, the winning approach modeled the matrix as having __rank 20 to 40.__

Source: [Koren et al, IEEE Computer, 2009]
::::


::: {.content-visible when-profile="slides"}
## Images
:::

::: {.content-visible when-profile="web"}
__Images.__

:::

Image data often shows low effective rank.

For example, here is an original photo:

In [ ]:
boat = np.loadtxt('data/images/boat/boat.dat')
import matplotlib.cm as cm
plt.figure()
plt.imshow(boat, cmap=cm.Greys_r)
plt.axis('off')
plt.show()

::: {.content-visible when-profile="slides"}
## Images continued
:::

Let's look at its spectrum:

In [ ]:
u, s, vt = np.linalg.svd(boat, full_matrices=False)
plt.plot(s)
plt.xlabel('$k$', size=16)
plt.ylabel(r'$\sigma_k$', size=16)
plt.title('Singular Values of Boat Image', size=16)
plt.show()

::: {.content-visible when-profile="slides"}
## Images continued
:::

This image is 512 $\times$ 512. As a matrix, it has rank of 512.   

But its _effective_ rank is low.

Based on the previous plot, its effective rank is perhaps 40.

Let's find the closest rank-40 matrix and view it.

In [ ]:
#| code-fold: false
u, s, vt = np.linalg.svd(boat, full_matrices=False)
s[40:] = 0
boatApprox = u @ np.diag(s) @ vt

In [ ]:
plt.figure(figsize=(9, 6))
plt.subplot(1, 2, 1)
plt.imshow(boatApprox, cmap=cm.Greys_r)
plt.axis('off')
plt.title('Rank 40 Boat')
plt.subplot(1, 2, 2)
plt.imshow(boat, cmap=cm.Greys_r)
plt.axis('off')
plt.title('Rank 512 Boat')
plt.show()

## Interpretations of Low Effective Rank

How can we understand the low-effective-rank phenomenon in general?

:::: {.fragment}
There are two helpful interpretations:

:::: {.incremental}
1. Common Patterns
2. Latent Factors
::::
::::

::: {.content-visible when-profile="slides"}
## Low Rank Implies Common Patterns
:::

::: {.content-visible when-profile="web"}
### Low Rank Implies Common Patterns
:::

The first interpretation of low-rank behavior is in answering the question:

:::: {.fragment}
"What is the strongest pattern in the data?"
::::

:::: {.fragment}
Remember that using the SVD we form the low-rank approximation as

$$ A^{(k)} =  U'\Sigma'(V')^T$$

and

:::: {.incremental}

- $U'$ are the $k$ leftmost columns of $U$, 
- $\Sigma'$ is the $k\times k$ upper left submatrix of $\Sigma$, and 
- $V'$ are the $k$ leftmost columns of $V$.
::::
::::
 
:::: {.fragment}
In this interpretation, we think of each column of $A^{(k)}$ as a combination of the columns of $U'$.
::::

:::: {.fragment}
How can this be helpful? 
::::


::: {.content-visible when-profile="slides"}
## Common Patterns Traffic Example
:::

Consider the set of traffic traces. There are clearly some common patterns. How can we find them?

In [ ]:
with open('data/net-traffic/AbileneFlows/odnames','r') as f:
    odnames = [line.strip() for line in f]
dates = pd.date_range('9/1/2003', freq='10min', periods=1008)
Atraf = pd.read_table('data/net-traffic/AbileneFlows/X', sep='  ', header=None, names=odnames, engine='python')
Atraf.index = dates
plt.figure(figsize=(10, 8))
for i in range(1, 13):
    ax = plt.subplot(4, 3, i)
    Atraf.iloc[:, i-1].plot()
    plt.title(odnames[i])
plt.subplots_adjust(hspace=1)
plt.suptitle('Twelve Example Traffic Traces', size=20)
plt.show()

::: {.content-visible when-profile="slides"}
## Common Patterns Traffic Example continued
:::

Let's use as our example $\mathbf{a}_1,$ the first column of $A$.

This happens to be the ATLA-CHIN flow.

The equation above tells us that

$$\mathbf{a}_1 \approx v_{11}\sigma_1\mathbf{u}_1 + v_{12}\sigma_2\mathbf{u}_2 + \dots + v_{1k}\sigma_k\mathbf{u}_k.$$

In other words, $\mathbf{u}_1$ (the first column of $U$) is the "strongest" pattern occurring in $A$, and its strength is measured by $\sigma_1$.

::: {.content-visible when-profile="slides"}
## Common Patterns Traffic Example continued
:::

Here is a view of the first 2 columns of $U\Sigma$ for the traffic matrix data.

These are the strongest patterns occurring across all of the 121 traces.

In [ ]:
u, s, vt = np.linalg.svd(Atraf, full_matrices=False)
uframe = pd.DataFrame(u @ np.diag(s), index=pd.date_range('9/1/2003', freq = '10min', periods = 1008))
uframe[0].plot(color='r', label='Column 1')
uframe[1].plot(label='Column 2')
plt.legend(loc='best')
plt.title('First Two Columns of $U$')
plt.show()

::: {.content-visible when-profile="slides"}
## Low Rank Defines Latent Factors
:::

::: {.content-visible when-profile="web"}
### Low Rank Defines Latent Factors
:::

The next interpretation of low-rank behavior is that it exposes "latent factors" that describe the data.

:::: {.fragment}
In this interpretation, we think of each element of $A^{(k)}=U'\Sigma'(V')^T$ as the inner product of a row of $U'\Sigma'$ and a row of $V'$.
::::

:::: {.fragment}
Let's say we are working with a matrix of users and items.
::::

:::: {.fragment}
In particular, let the items be movies and matrix entries be ratings, as in the Netflix prize.
::::

::: {.content-visible when-profile="slides"}
## Low Rank Defines Latent Factors Netflix example
:::

Recall the structure from earlier:

$$
\begin{bmatrix}
\vdots & \vdots &  & \vdots \\
\mathbf{a}_{1} & \mathbf{a}_{2} & \cdots & \mathbf{a}_{n} \\
\vdots & \vdots &  & \vdots \\
\end{bmatrix} 
\approx
\begin{bmatrix}
\vdots &  \vdots  \\
\sigma_1 \mathbf{u}_1 & \sigma_k \mathbf{u}_{k} \\
\vdots& \vdots  \\
\end{bmatrix}
\begin{bmatrix}
\cdots & \mathbf{v}_1 & \cdots   \\
\cdots & \mathbf{v}_k & \cdots   \\
\end{bmatrix},
$$

where the rows of $A$ are the users and the columns are movie ratings.

Then the rating that a user gives a movie is the inner product of a $k$ element vector that corresponds to the user, and a $k$ element vector that corresponds to the movie.

In other words:
    
$$ a_{ij} = \tilde{\mathbf{u}}_i^T \mathbf{v}_j.$$

::: {.content-visible when-profile="slides"}
## Netflix example
:::

We can therefore think of user $i$'s preferences as being captured by $\tilde{\mathbf{u}}_i = \sigma_i\mathbf{u}_i$, i.e., a point in $\mathbb{R}^k$.  

:::: {.fragment}
We have described everything we need to know to predict user $i$'s ratings via a $k$-element vector.
::::

:::: {.fragment}
The $k$-element vector is called a __latent factor.__
::::

:::: {.fragment}
Likewise, we can think of $\mathbf{v}_j$ as a "description" of movie $j$ (another latent factor).
::::

:::: {.fragment}
The value in using latent factors comes from the summarization of user preferences, and the predictive power one obtains.
::::

:::: {.fragment}
For example, the winning entry in the Netflix prize competition modeled user preferences with a 20-element latent factor.
::::

:::: {.fragment}
The remarkable thing is that a person's preferences for all 18,000 movies can be reasonably well captured in a 20-element vector!
::::

::: {.content-visible when-profile="slides"}
## Netflix example continued
:::

Here is a figure from the paper that described the winning strategy in the Netflix prize.

It shows a hypothetical __latent space__ in which each user, and each movie, is represented by a latent vector.

![](figs/L10-Movie-Latent-Space.png)

Source: Koren et al, IEEE Computer, 2009 

In practice, this is perhaps a 20- or 40-dimensional space.

::: {.content-visible when-profile="slides"}
## Netflix example continued
:::

Here are some representations of movies in that space (reduced to 2-D).

Notice how the space seems to capture similarity among movies!

![](figs/L10-Netflix-Latent-Factors.png)

Source: Koren et al, IEEE Computer, 2009 

## Summary

:::: {.incremental}
* When we are working with data matrices, it is valuable to consider the __effective rank__.
* Many (many) datasets in real life show __low effective rank__.
* This property can be explored precisely using the Singular Value Decomposition of the matrix.
* When low effective rank is present
    * the matrix can be compressed with only small loss of accuracy,
    * we can extract the "strongest" patterns in the data,
    * we can describe each data item in terms of the inner product of __latent factors__.
::::
